In [ ]:
#import pandas
import pandas as pd
#read in data from "births.csv" and "score.csv" as dataframes
birthsdf=pd.read_csv("files/births.csv", sep=",")
scoredf=pd.read_csv("files/score.csv", sep=",")

In [16]:
#examine the births dataset
birthsdf.head(10)
print(birthsdf.dtypes)

id           int64
bweight      int64
lowbw        int64
gestwks    float64
preterm    float64
matage       int64
hyp          int64
sex          int64
dtype: object


In [19]:


birthsdf["lowbw"]=pd.Categorical(birthsdf["lowbw"], categories=[0,1])
print(birthsdf.dtypes)


id            int64
bweight       int64
lowbw      category
gestwks     float64
preterm     float64
matage        int64
hyp           int64
sex           int64
dtype: object
